# Data Sparsity Check

In [2]:
import pandas as pd

In [3]:
g = pd.read_pickle ('../data/interim/X_group.pkl')
t = pd.read_pickle ('../data/interim/X_technique.pkl')
y = pd.read_pickle ('../data/interim/y_cleaned.pkl')

In [4]:
t

,technique_ID,input_technique_platforms,input_technique_tactics,input_technique_data_sources,input_technique_defenses_bypassed,input_technique_permissions_required,input_technique_mitigation_id,input_technique_detection_name,input_technique_software_id,input_technique_description
0,T1548,"[linux, windows, macos]","[defense_evasion, privilege_escalation]","[command_command_execution, file_file_metadata...",[],"[administrator, user]","[m1047, m1038, m1028, m1026, m1022, m1052]","[command_execution, file_metadata, file_modifi...",[],"[-0.36630338, -0.24036503, -0.36260876, 0.0984..."
1,T1548.002,[windows],"[defense_evasion, privilege_escalation]","[command_command_execution, process_process_cr...",[windows_user_account_control],"[administrator, user]","[m1047, m1026, m1051, m1052]","[command_execution, process_creation, process_...","[s0584, s0129, s0640, s0606, s0570, s1068, s00...","[-0.5305501, -0.43114784, -0.67361057, 0.16222..."
2,T1548.004,[macos],"[defense_evasion, privilege_escalation]","[process_os_api_execution, process_process_cre...",[],"[administrator, user]",[m1038],"[os_api_execution, process_creation]",[s0402],"[-0.57463825, -0.65824693, -0.23797776, -0.254..."
3,T1548.001,"[linux, macos]","[defense_evasion, privilege_escalation]","[command_command_execution, file_file_metadata...",[],[user],[m1028],"[command_execution, file_metadata, file_modifi...","[s0401, s0276]","[-0.7420097, -0.461457, -0.45979652, -0.116038..."
4,T1548.003,"[linux, macos]","[defense_evasion, privilege_escalation]","[command_command_execution, file_file_modifica...",[],[user],"[m1028, m1026, m1022]","[command_execution, file_modification, process...","[s0154, s0281, s0279]","[-0.8037999, -0.6815256, 0.18332177, 0.0635958..."
...,...,...,...,...,...,...,...,...,...,...
602,T1102.002,"[linux, windows, macos]",[command_and_control],"[network_traffic_network_connection_creation, ...",[],[user],"[m1031, m1021]","[network_connection_creation, network_traffic_...","[s0128, s0069, s0657, s0651, s0025, s0660, s00...","[0.0067764707, -0.35248113, -0.62356573, 0.294..."
603,T1102.001,"[linux, windows, macos]",[command_and_control],"[network_traffic_network_traffic_content, netw...",[],[user],"[m1031, m1021]","[network_traffic_content, network_traffic_flow]","[s0373, s0128, s0069, s0674, s0531, s0528, s10...","[-0.7252811, -0.3269149, -0.49607247, 0.198911..."
604,T1102.003,"[linux, windows, macos]",[command_and_control],"[network_traffic_network_connection_creation, ...",[],[user],"[m1031, m1021]","[network_connection_creation, network_traffic_...","[s0568, s0037, s0455, s0052]","[-0.11939225, -0.35763258, -0.46435547, 0.5083..."
605,T1047,[windows],[execution],"[command_command_execution, network_traffic_ne...",[],[],"[m1040, m1038, m1026, m1018]","[command_execution, network_connection_creatio...","[s1028, s0331, s0373, s0640, s0534, s1070, s10...","[-0.63444924, -0.15962608, -0.2825894, 0.07326..."


In [5]:
from statistics import mode
def get_feature_length_stats(df: pd.DataFrame()):
    for col in list (df.columns)[1:]:
        list_lengths = df[col].apply(lambda x: len(x))
        avg_len = list_lengths.mean()
        mode_len = mode(list_lengths)
        max_len = list_lengths.max()
        print ('{col} \tmean_len: {mean_len}\tmode_len: {mode_len}\tmax_len: {max_len}'.format(col = col, mean_len= avg_len, mode_len = mode_len, max_len = max_len))
        
def get_vocab_size (df: pd.DataFrame):
    for feature_name in df.columns[1:]:
        vocab_size = len(df[feature_name].explode().unique())
        print ('{feature_name}: vocab size = {vocab_size}'.format (feature_name = feature_name, vocab_size = vocab_size))

In [6]:
def get_emtpy_instance_percentage (df, feature_name):
    value_counts = df[feature_name].value_counts()
    blank_list_count = 0
    for value,count in value_counts.items():
        if value == ['']: 
            blank_list_count = count
            break
    print ("empty instance percentage: ", blank_list_count/ len(df) *100)


In [7]:
for feature_name in t.columns[1:-1]:
    print ('-----', feature_name)
    get_emtpy_instance_percentage (t, feature_name)

----- input_technique_platforms
empty instance percentage:  0.0
----- input_technique_tactics
empty instance percentage:  0.0
----- input_technique_data_sources
empty instance percentage:  6.425041186161449
----- input_technique_defenses_bypassed
empty instance percentage:  83.03130148270182
----- input_technique_permissions_required
empty instance percentage:  66.88632619439868
----- input_technique_mitigation_id
empty instance percentage:  15.32125205930807
----- input_technique_detection_name
empty instance percentage:  6.425041186161449
----- input_technique_software_id
empty instance percentage:  33.113673805601316


In [8]:
for feature_name in g.columns[1:-1]:
    print ('-----', feature_name)
    get_emtpy_instance_percentage (g, feature_name)

----- input_group_software_id
empty instance percentage:  8.823529411764707


Conclusion:
- `input_technique_defenses_bypassed` and `input_technique_permissions_required` contains more than 50% empty instances. These feature will NOT be used in training
---

In [9]:
for col in t.columns[1:-1]:
    print ('--------',col)
    feature_name = col
    print (t.explode(feature_name)[feature_name].value_counts())

-------- input_technique_platforms
input_technique_platforms
windows             421
macos               303
linux               295
pre                  88
iaas                 84
office_365           72
network              65
saas                 58
google_workspace     56
azure_ad             46
containers           36
Name: count, dtype: int64
-------- input_technique_tactics
input_technique_tactics
defense_evasion         184
persistence             113
privilege_escalation     96
credential_access        63
resource_development     45
discovery                44
reconnaissance           43
command_and_control      39
collection               37
execution                36
impact                   26
lateral_movement         22
initial_access           19
exfiltration             18
Name: count, dtype: int64
-------- input_technique_data_sources
input_technique_data_sources
command_command_execution                  268
process_process_creation                   226
network_traff